In [1]:
%matplotlib inline

In [2]:
# import default scientific libraries
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [3]:
# import pandas for easy data management, keras for deep learning and scikit for feature generation
import pandas as pd
import keras
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko
import mca
from sklearn import metrics, model_selection, manifold, decomposition

Using TensorFlow backend.


In [ ]:
# load needed feature

In [4]:
items = np.load('items.npy')
items.shape

(10000, 128)

In [5]:
items2 = np.load('items2.npy')
items2.shape

(10000, 128)

In [6]:
items3 = np.load('items3.npy')
items3.shape

(10000, 56)

In [7]:
users = np.load('users.npy')
users.shape

(1000, 128)

In [8]:
users2 = np.load('users2.npy')
users2.shape

(1000, 128)

In [9]:
users3 = np.load('users3.npy')
users3.shape

(1000, 97)

In [10]:
# ensure tensorflow does not leak
keras.backend.clear_session()

In [ ]:
# load model

In [11]:
model1 = km.load_model('model1.h5')

In [12]:
# compute weighted mean between with rating confidence and rating
def weighted_mean(preds):
    ret = []
    for e, s in zip(preds, np.argsort(preds, axis=1)):
        # highest confidence rating (index of the sorted array)
        #               |
        #               v
        n1, n2, n3, n4, n5 = s
        val = (n5 * e[n5] + n4 * e[n4] + n3 * e[n3] + n2 * e[n2] + n1 * e[n1]) / (e[n1] + e[n2] + e[n3] + e[n4] + e[n5])
        ret.append(val + 1)
    return np.array(ret)

In [13]:
# load submission data
sub = pd.read_csv('sampleSubmission.csv')
pos = sub.Id.str.extract('r([0-9]+)_c([0-9]+)', expand=True)
sub['User'] = pos[0].astype(np.int)
sub['Item'] = pos[1].astype(np.int)
sub.head()

,Id,Prediction,User,Item
0,r37_c1,3,37,1
1,r73_c1,3,73,1
2,r156_c1,3,156,1
3,r160_c1,3,160,1
4,r248_c1,3,248,1


In [14]:
# compute prediction through the two network
pred2 = weighted_mean(model1.predict([sub.Item, sub.User, users[sub.Item - 1], items[sub.User - 1], users2[sub.Item - 1], items2[sub.User - 1], users3[sub.Item - 1], items3[sub.User - 1]], batch_size=4096))

In [15]:
# store prediction and output csv formatted for submission
sub['Prediction'] = pred2
sub.head()
sub.to_csv('submission.csv', columns=['Id', 'Prediction'], index=False)